In [ ]:
# This notebook is an example notebook for how to do a slew and settle test

In [ ]:
# As a precondition, and not part of this notebook, the telescope system should be fully enabled and configured.

In [ ]:
import sys
import asyncio
import time
import os
from datetime import datetime

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [ ]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#get catcs.rem.atmcs.evt_summaryState
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
tmp = await atcs.rem.atmcs.evt_summaryState.aget()
print(tmp)

In [ ]:
salobj.State(2)

In [ ]:
# enable components if required
# await atcs.standby()
# await latiss.standby()

In [ ]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
# turn on ATAOS corrections just to make sure the mirror is under air
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=False)

In [ ]:
# slew telescope to desired starting position
# rotator does not move for this test as it's part of a different
# requirement/verification
start_az=0
start_el=80
start_rot_pa=0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
# point telescope to desired starting position
start_az=0
start_el=70
start_rot_pa=0

In [ ]:
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
#declare offset sizes for tests

# For small slew and settle
d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

# For large slew and settle
d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

# What we actually care about for the survey! So measure this too
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

In [ ]:
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
# get RA/DEC of current telescope Alt/Az position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)
# This should be TAI and not UTC... so will be 37s off system clock seconds
curr_time_atptg = Time(time_data.mjd, format="mjd")

coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))

In [ ]:
#send to starting position
# Note that the wait_settle=False is used to remove sleeps
# that are no longer required in the ATCS now that ATMCS

await attcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot_pa=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image to get some header data
await latiss.take_engtest(exptime=1)

print('Starting to Slew to target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                      rot_pa=d_rot, slew_timeout=240., 
                      stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

# Print current time so it's easier to find info in EFD
current_time = datetime.now().strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
# Now you'll want run the test but going to the original position!

print('Starting to Slew to original target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                      rot_pa=d_rot, slew_timeout=240., 
                      stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

# Print current time so it's easier to find info in EFD
current_time = datetime.now().strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
# For shutdown of system
await attcs.stop_tracking()

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)